In [1]:
from os import listdir
from os.path import isfile, join
import re
from typing import List

import pandas as pd
import numpy as np

In [20]:
path = "all_data/"
filepaths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
r = re.compile(".*VAERSDATA.csv")
fp_data = list(filter(r.match, filepaths))

In [21]:
fp_data

['all_data/1993VAERSDATA.csv',
 'all_data/1994VAERSDATA.csv',
 'all_data/2000VAERSDATA.csv',
 'all_data/2007VAERSDATA.csv',
 'all_data/2012VAERSDATA.csv',
 'all_data/2015VAERSDATA.csv',
 'all_data/1995VAERSDATA.csv',
 'all_data/1992VAERSDATA.csv',
 'all_data/2014VAERSDATA.csv',
 'all_data/2013VAERSDATA.csv',
 'all_data/2006VAERSDATA.csv',
 'all_data/2001VAERSDATA.csv',
 'all_data/2019VAERSDATA.csv',
 'all_data/1998VAERSDATA.csv',
 'all_data/2018VAERSDATA.csv',
 'all_data/NonDomesticVAERSDATA.csv',
 'all_data/1999VAERSDATA.csv',
 'all_data/2008VAERSDATA.csv',
 'all_data/2009VAERSDATA.csv',
 'all_data/1990VAERSDATA.csv',
 'all_data/1997VAERSDATA.csv',
 'all_data/2020VAERSDATA.csv',
 'all_data/2011VAERSDATA.csv',
 'all_data/2016VAERSDATA.csv',
 'all_data/2003VAERSDATA.csv',
 'all_data/2004VAERSDATA.csv',
 'all_data/2021VAERSDATA.csv',
 'all_data/1996VAERSDATA.csv',
 'all_data/1991VAERSDATA.csv',
 'all_data/2005VAERSDATA.csv',
 'all_data/2002VAERSDATA.csv',
 'all_data/2017VAERSDATA.csv',
 

In [24]:
df = pd.DataFrame()
for path in fp_data:
    df_tmp = pd.read_csv(path, low_memory=False)
    df = df.append(df_tmp)
    
df.shape

(1136617, 35)

## convert dates to datetime

In [25]:
df['RECVDATE'] = pd.to_datetime(df['RECVDATE'], format="%m/%d/%Y")
df['DATEDIED'] = pd.to_datetime(df['DATEDIED'], format="%m/%d/%Y")
df['VAX_DATE'] = pd.to_datetime(df['VAX_DATE'], format="%m/%d/%Y")
df['ONSET_DATE'] = pd.to_datetime(df['ONSET_DATE'], format="%m/%d/%Y")
df['TODAYS_DATE'] = pd.to_datetime(df['TODAYS_DATE'], format="%m/%d/%Y")

## replace nan with 'N' where applicable

In [26]:
def convert_date_format(df: pd.DataFrame, cols: List[str], date_format: str) -> pd.DataFrame:
    for col in cols:
        df[col] = pd.to_datetime(df[col], format=date_format)
    return df

In [27]:
def convert_nans(df: pd.DataFrame, cols: List[str], replace_value:str) -> pd.DataFrame:
    for col in cols:
        df[col].replace(np.nan, replace_value, inplace=True)
    return df

In [28]:
cols_date_convert = ['RECVDATE', 'DATEDIED', 'VAX_DATE', 'ONSET_DATE', 'TODAYS_DATE']
cols_nans_convert = ['DIED', 'HOSPITAL']
date_format = "%m/%d/%Y"

In [29]:
df = convert_date_format(df, cols_date_convert, date_format)
df = convert_nans(df, cols_nans_convert, 'N')

In [30]:
cut_bins = [0, 15, 25, 35, 45, 55, 65, 75, 85, 120]
labels = ["0-15", "15-25", "25-35", "35-45", "45-55", "55-65", 
         "65-75", "75-85", ">85"]

In [31]:
df['AGE_BIN'] = pd.cut(df['AGE_YRS'], bins=cut_bins)

In [32]:
pd.cut(df['AGE_YRS'], bins=cut_bins)[0]

0     (0.0, 15.0]
0    (35.0, 45.0]
0     (0.0, 15.0]
0    (35.0, 45.0]
0    (65.0, 75.0]
0     (0.0, 15.0]
0     (0.0, 15.0]
0    (25.0, 35.0]
0    (35.0, 45.0]
0             NaN
0    (15.0, 25.0]
0     (0.0, 15.0]
0    (65.0, 75.0]
0    (75.0, 85.0]
0    (65.0, 75.0]
0    (75.0, 85.0]
0    (75.0, 85.0]
0    (25.0, 35.0]
0             NaN
0     (0.0, 15.0]
0    (25.0, 35.0]
0    (45.0, 55.0]
0     (0.0, 15.0]
0    (65.0, 75.0]
0     (0.0, 15.0]
0    (25.0, 35.0]
0    (25.0, 35.0]
0     (0.0, 15.0]
0     (0.0, 15.0]
0    (55.0, 65.0]
0    (55.0, 65.0]
0     (0.0, 15.0]
0    (25.0, 35.0]
Name: AGE_YRS, dtype: category
Categories (9, interval[int64]): [(0, 15] < (15, 25] < (25, 35] < (35, 45] ... (55, 65] < (65, 75] < (75, 85] < (85, 120]]

In [33]:
df['RECVYEAR'] = df['RECVDATE'].dt.year

In [34]:
df['RECVYEAR'].unique()

array([1993, 1994, 2000, 2007, 2012, 2015, 1995, 1992, 2014, 2013, 2006,
       2001, 2019, 1998, 2018, 1990, 1991, 1996, 1997, 1999, 2002, 2003,
       2004, 2005, 2008, 2009, 2010, 2011, 2016, 2017, 2020, 2021])

In [38]:
df_yr = df.groupby(['RECVYEAR', 'AGE_BIN'])['VAERS_ID'].count().reset_index()

In [43]:
df_mean_prev = df_yr[df_yr['RECVYEAR']<2021].groupby(['AGE_BIN']).mean()
df_mean_prev.columns = ['yr','mean_incidents']
df_mean_prev

,yr,mean_incidents
AGE_BIN,,
"(0, 15]",2005.0,9290.612903
"(15, 25]",2005.0,2087.483871
"(25, 35]",2005.0,1634.064516
"(35, 45]",2005.0,1544.806452
"(45, 55]",2005.0,1688.612903
"(55, 65]",2005.0,2192.290323
"(65, 75]",2005.0,2043.967742
"(75, 85]",2005.0,774.096774
"(85, 120]",2005.0,172.258065


In [44]:
df_yr[df_yr['RECVYEAR']==2021]

,RECVYEAR,AGE_BIN,VAERS_ID
279,2021,"(0, 15]",3124
280,2021,"(15, 25]",26613
281,2021,"(25, 35]",44726
282,2021,"(35, 45]",51842
283,2021,"(45, 55]",50376
284,2021,"(55, 65]",51554
285,2021,"(65, 75]",43734
286,2021,"(75, 85]",18189
287,2021,"(85, 120]",6380


## create subset

In [45]:
df.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES,AGE_BIN,RECVYEAR
0,48573,1993-01-04,AR,0.5,0.0,0.5,M,08/12/1992,pt recvd vax 11AUG92 4PM & exp hives over uppe...,N,...,NONE~ ()~~~In patient,AR9248,1,NaT,NaN,NaN,NaN,NaN,"(0, 15]",1993
1,48574,1993-01-04,AR,13.0,13.0,NaN,F,08/18/1992,"p/inject pt became nauseated & felt sick, like...",N,...,~ ()~~~In patient,AR9249,1,NaT,NaN,NaN,NaN,NaN,"(0, 15]",1993
2,48575,1993-01-04,AR,4.0,4.0,NaN,F,08/25/1992,TC from mom had 1 bump or 2 on face a/got shot...,N,...,NA~ ()~~~In patient,AR9250,1,NaT,NaN,NaN,NaN,NaN,"(0, 15]",1993
3,48576,1993-01-04,AR,5.0,5.0,NaN,M,08/26/1992,"mom describes pt fell to floor trembled, did n...",N,...,NA~ ()~~~In patient,AR9251,1,NaT,NaN,NaN,NaN,NaN,"(0, 15]",1993
4,48577,1993-01-04,AR,1.1,1.0,0.1,M,08/26/1992,"fine red, bumpy rash around inject site notice...",N,...,NONE~ ()~~~In patient,AR9252,1,NaT,NaN,NaN,NaN,NaN,"(0, 15]",1993


In [48]:
df_yr_died = df.groupby(['RECVYEAR', 'DIED'])['VAERS_ID'].count()
df_yr_died.head()

RECVYEAR  DIED
1990      N        2071
          Y          80
1991      N        9826
          Y         166
1992      N       10589
Name: VAERS_ID, dtype: int64

In [53]:
df_died_perc = df_yr_died.groupby(level=0).apply(lambda x: x * 100 / float(x.sum())).reset_index()

In [54]:
df_died_perc.head()

,RECVYEAR,DIED,VAERS_ID
0,1990,N,96.280800
1,1990,Y,3.719200
2,1991,N,98.338671
3,1991,Y,1.661329
4,1992,N,97.892207


In [62]:
df_died_perc[(df_died_perc['DIED']=="Y")]

,RECVYEAR,DIED,VAERS_ID
1,1990,Y,3.719200
3,1991,Y,1.661329
5,1992,Y,2.107793
7,1993,Y,2.270081
9,1994,Y,2.288749
11,1995,Y,1.538012
13,1996,Y,1.349540
15,1997,Y,1.490351
17,1998,Y,1.595251
19,1999,Y,1.389967


In [58]:
df_died_perc[(df_died_perc['RECVYEAR']<2021) & (df_died_perc['DIED']=="Y")].groupby('DIED').mean()

,RECVYEAR,VAERS_ID
DIED,,
Y,2005,1.332726


In [61]:
df_died_perc[(df_died_perc['RECVYEAR']==2021)]

,RECVYEAR,DIED,VAERS_ID
62,2021,N,98.123416
63,2021,Y,1.876584
